# Importing Libraries

In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import random
import zipfile
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import confusion_matrix
import json
from numba import cuda 

In [11]:
#Used for local GPU usage
cuda.select_device(0)
cuda.close()

In [2]:
#Setting seeds for replicating similar results
seed = 42
random.seed(42)
np.random.seed(seed=seed)
tf.random.set_seed(seed=seed)

In [3]:
# train_zip = "dogs-vs-cats-redux-kernels-edition/train.zip"
# zip_ref = zipfile.ZipFile(train_zip,"r")
# zip_ref.extractall("working/")

In [4]:
# test_zip = "dogs-vs-cats-redux-kernels-edition/test.zip"
# zip_ref = zipfile.ZipFile(test_zip,"r")
# zip_ref.extractall("working/test/")

In [5]:
#Defining Paths for working, train, validation and test directories
work = "working/"
train_path = work + "train/"
valid_path = work + "valid/"
test_path = work + "test/"

In [6]:
#Defining cat and dog paths for train and validation sets
dog_train_path = train_path + "dog/"
cat_train_path = train_path + "cat/"
dog_valid_path = valid_path + "dog/"
cat_valid_path = valid_path + "cat/"

In [ ]:
#Code used to create images in the paths defined to use Tensorflows Flow from directory
try: 
    os.mkdir(valid_path)
    os.mkdir(dog_train_path)
    os.mkdir(cat_train_path)
    os.mkdir(dog_valid_path)
    os.mkdir(cat_valid_path)
except:
  print("dir already exists")    

cat_files = [img for img in os.listdir(train_path) if img[:3] == "cat"]
dog_files = [img for img in os.listdir(train_path) if img[:3] == "dog"]

cat_files_train = random.sample(cat_files, k=round(len(cat_files) * 0.8))
dog_files_train = random.sample(dog_files, k=round(len(dog_files) * 0.8))

cat_files_valid = list(set(cat_files) - set(cat_files_train))
dog_files_valid = list(set(dog_files) - set(dog_files_train))

[os.rename(train_path + file, dog_train_path + file) for file in dog_files_train]
[os.rename(train_path + file, dog_valid_path + file) for file in dog_files_valid]
[os.rename(train_path + file, cat_train_path + file) for file in cat_files_train]
[os.rename(train_path + file, cat_valid_path + file) for file in cat_files_valid]

In [7]:
#Defining Batch size, augmentation levels, class num image target size
batch_size = 32
augment_value = 0.05
num_classes = 2
img_size = (299, 299)

In [8]:
#Augmenting and Importing Data using ImageDataGenerator and Flow From Directory modules
    #from Tensorflow for both training and validation datasets
train_datagen = ImageDataGenerator(
    rescale = 1. / 255,
    rotation_range = augment_value,
    height_shift_range = augment_value,
    width_shift_range = augment_value,
    shear_range = augment_value,
    zoom_range = augment_value,
    cval = augment_value,
    horizontal_flip = True,
    vertical_flip = True
)

validation_datagen = ImageDataGenerator(
    rescale = 1. / 255
)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size = img_size,
    batch_size = batch_size,
    class_mode = 'categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    valid_path,
    target_size = img_size,
    batch_size = batch_size,
    class_mode = 'categorical'
)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [9]:
#Similarly importing test data using flow from directory
test_datagen = ImageDataGenerator(
    rescale = 1. / 255
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size = img_size,
    batch_size = batch_size,
    class_mode = None,
    shuffle = False
)

Found 12500 images belonging to 1 classes.


In [ ]:
#Creating a custom CNN Classifier
with tf.device('/cpu:0'):
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(64, 7, activation = "relu", padding = "same", 
                                  input_shape = [299, 299, 3], strides = (1, 1)))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Conv2D(64, 7, activation = "relu", padding = "same", strides = (1, 1)))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.MaxPool2D(2))
    model.add(keras.layers.Dropout(0.25))
    model.add(keras.layers.Conv2D(64, 3, activation = "relu", padding = "same", strides = (1, 1)))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Conv2D(64, 3, activation = "relu", padding = "same", strides = (1, 1)))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.MaxPool2D(2))
    model.add(keras.layers.Dropout(0.25))
    model.add(keras.layers.Conv2D(64, 3, activation = "relu", padding = "same", strides = (1, 1)))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Conv2D(64, 3, activation = "relu", padding = "same", strides = (1, 1)))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.MaxPool2D(2))
    model.add(keras.layers.Dropout(0.25))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(128, activation="relu"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(64, activation="relu"))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(2, activation="softmax"))
#I got a maximum accuracy of 0.85 with this model

In [10]:
#Defining model checkpoint and earlystopping callbacks to retrieve the best
    #parameters after the validation accuracy stops improving
callbacks_list = [
    ModelCheckpoint(work, monitor = 'val_accuracy', verbose = 1, save_best_only = True),
    EarlyStopping(monitor = 'val_accuracy', patience = 2, verbose = 0)
]

In [ ]:
#Defining the optimizer and compiling the model
sgd = keras.optimizers.SGD(learning_rate = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])

In [ ]:
#Training the model, 
history = model.fit(
        train_generator,
        validation_data = validation_generator,
        epochs = 1000,
        steps_per_epoch = int(np.ceil(20000/batch_size)),
        validation_steps = int(np.ceil(5000/batch_size)),
        callbacks = callbacks_list
)

Using Transfer Learning

In [11]:
#Using Transfer Learning using the ResnetInceptionV2 Model
with tf.device('/gpu:0'):
    base_model = keras.applications.InceptionResNetV2(weights = "imagenet",
                                                      include_top = False)
    avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
    dense = keras.layers.Dense(512, activation = "relu")(avg)
    bn = keras.layers.BatchNormalization()(dense)
    drop = keras.layers.Dropout(.2)(bn)
    output = keras.layers.Dense(num_classes, activation = "sigmoid")(drop)
    model = keras.models.Model(inputs = base_model.input, outputs = output)

In [10]:
#Training the last layers of the model without changing the original model
for layer in base_model.layers:
    layer.trainable = False

sgd = keras.optimizers.SGD(learning_rate = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)

model.compile(loss = "categorical_crossentropy", optimizer = sgd,
              metrics = ["accuracy"])

history = model.fit(
        train_generator,
        validation_data = validation_generator,
        epochs = 30,
        steps_per_epoch = int(np.ceil(20000/batch_size)),
        validation_steps = int(np.ceil(5000/batch_size)),
        callbacks = callbacks_list
)


Epoch 1/30
625/625 [==============================] - ETA: 0s - loss: 0.1126 - accuracy: 0.9627
Epoch 1: val_accuracy improved from -inf to 0.99340, saving model to working\
INFO:tensorflow:Assets written to: working\assets
625/625 [==============================] - 505s 791ms/step - loss: 0.1126 - accuracy: 0.9627 - val_loss: 0.0242 - val_accuracy: 0.9934
Epoch 2/30
625/625 [==============================] - ETA: 0s - loss: 0.0874 - accuracy: 0.9686
Epoch 2: val_accuracy did not improve from 0.99340
625/625 [==============================] - 459s 734ms/step - loss: 0.0874 - accuracy: 0.9686 - val_loss: 0.0271 - val_accuracy: 0.9912
Epoch 3/30
625/625 [==============================] - ETA: 0s - loss: 0.0847 - accuracy: 0.9703
Epoch 3: val_accuracy improved from 0.99340 to 0.99420, saving model to working\
INFO:tensorflow:Assets written to: working\assets
625/625 [==============================] - 512s 819ms/step - loss: 0.0847 - accuracy: 0.9703 - val_loss: 0.0196 - val_accuracy: 0.99

In [ ]:
model.save(work + "model_InceptionResNetV2_final")

In [ ]:
del(base_model)
del(model)
tf.keras.backend.clear_session()

In [12]:
with tf.device('/cpu:0'):
    model = tf.keras.models.load_model(work + "model_InceptionResNetV2_final")

In [13]:
#Training the entire model as the final layers have initialized earlier
for layer in base_model.layers:
    layer.trainable = True

sgd = keras.optimizers.SGD(learning_rate = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
    
model.compile(loss = "categorical_crossentropy", optimizer = sgd,
              metrics = ["accuracy"])
with tf.device('/gpu:0'):
    history_2 = model.fit(
            train_generator,
            validation_data = validation_generator,
            epochs = 30,
            steps_per_epoch = int(np.ceil(20000/batch_size)),
            validation_steps = int(np.ceil(5000)/batch_size),
            callbacks = callbacks_list
    )

Epoch 1/30
625/625 [==============================] - ETA: 0s - loss: 0.0650 - accuracy: 0.9776
Epoch 1: val_accuracy improved from -inf to 0.99479, saving model to working\
INFO:tensorflow:Assets written to: working\assets
625/625 [==============================] - 529s 829ms/step - loss: 0.0650 - accuracy: 0.9776 - val_loss: 0.0171 - val_accuracy: 0.9948
Epoch 2/30
625/625 [==============================] - ETA: 0s - loss: 0.0585 - accuracy: 0.9794
Epoch 2: val_accuracy did not improve from 0.99479
625/625 [==============================] - 475s 760ms/step - loss: 0.0585 - accuracy: 0.9794 - val_loss: 0.0237 - val_accuracy: 0.9926
Epoch 3/30
625/625 [==============================] - ETA: 0s - loss: 0.0587 - accuracy: 0.9780
Epoch 3: val_accuracy did not improve from 0.99479
625/625 [==============================] - 478s 765ms/step - loss: 0.0587 - accuracy: 0.9780 - val_loss: 0.0228 - val_accuracy: 0.9932


In [14]:
model.save(work + "model_InceptionResNetV2_final_full")

INFO:tensorflow:Assets written to: working/model_InceptionResNetV2_final_full\assets


In [15]:
f = open('model_loss.json')
model_loss = json.load(f)

In [16]:
model_loss["model_InceptionResNetV2_final_full"] = min(history_2.history["loss"])

In [17]:
with open('model_loss.json', 'w') as f:
    json.dump(model_loss, f)

In [18]:
#Checking losses of all the models trained by transfer learning

In [19]:
model_loss

{'InceptionResNetV2': 0.0615997314453125,
 'model_NASNetLarge': 0.04065510258078575,
 'model_ResNet152V2': 0.05378391593694687,
 'model_MobileNetV3Large_Full': 0.6385003924369812,
 'model_NASNetLarge_SGD_full': 0.017551414668560028,
 'model_InceptionResNetV2_SGD_full': 0.04393969848752022,
 'model_InceptionResNetV2_SGD_full_0.2': 0.060112982988357544,
 'model_Xception_SGD_full_0.2': 0.05441661924123764,
 'model_InceptionResNetV2_SGD_half_0.1_bn': 0.0679125040769577,
 'model_InceptionV3_SGD_half_0.1_bn': 0.07481814175844193,
 'model_InceptionResNetV2_final_full': 0.05845284461975098}

In [ ]:
with tf.device('/gpu:0'):
    model = tf.keras.models.load_model(work + "model_InceptionResNetV2_final_full")

In [ ]:
#Predicting on the test images and creating the submission file

In [20]:
pred = model.predict(test_generator)
y_pred = [x[1] for x in pred]
y_pred = np.array(y_pred)
ids = test_generator.filenames
test_x = np.sort([idx.split("\\")[1].split(".")[0] for idx in ids])
data_1 = pd.DataFrame({'id': test_x, 'label': y_pred}, columns=['id', 'label'])
data_1["ids_sorted"] = pd.to_numeric(data_1["id"])
data_1.sort_values(by = ["ids_sorted"], inplace=True)
data_1.drop(["ids_sorted"], axis = 1, inplace=True)

In [22]:
data_1.to_csv("model_InceptionResNetV2_final_full.csv", index=False)

I got a final score of 0.044 logloss on Kaggle, which puts me in top 20 rankings.

In [ ]:
##Creating functions for kaggle
# import zipfile

# train_zip='/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip'
# zip_ref=zipfile.ZipFile(train_zip,'r')
# zip_ref.extractall('/kaggle/working/')


# work='/kaggle/working'
# train_path='/kaggle/working/train/'
# file_items=os.listdir(train_path)